In [ ]:
import sqlite3
con = sqlite3.connect('../data/GuidePod.sqlite')

In [ ]:
def getfeedURL(prmPodcasts):
    import json
    from urllib.request import urlopen
    import csv
    import re 
    #https://itunes.apple.com/lookup?id=1135137367&country=US&media=podcast&entity=podcastEpisode&limit=1
    feedURLs = []
    for eachpodcast in prmPodcasts:
        country = eachpodcast[0]
        pod_id = eachpodcast[1]
        dfeeds = []
        url = "https://itunes.apple.com/lookup?id=" + \
                str(pod_id) + \
                "&country=" + country + \
                "&media=podcast&entity=podcastEpisode&limit=0"
        
        try:
            with urlopen(url) as data_file:    
                    reviewdata = json.load(data_file)
                    reviewdataresults = reviewdata["results"] 
                    feedUrl = reviewdataresults[0]["feedUrl"]

            dfeeds = [pod_id, country, feedUrl]        
        except Exception as err:
            print (err)
            pass
        
        feedURLs.append(dfeeds) 

    return feedURLs

In [ ]:
def selectRows():
    import sqlite3
    con = sqlite3.connect('../data/GuidePod.sqlite')
    cur = con.cursor()
    all_pods = []
    try:
        sql = '''select distinct id, country, feedurl, name, Primary_Genre, podcast_id from podcast_master pd ''' + \
              '''left join episode_detail ed on pd.id = ed.podcast_id where podcast_id is null'''
        
        rows = cur.execute(sql)
    except Exception as err:
        print (err)
        pass
    #con.commit()
    for row in rows:
        all_pods.append([row[0], row[1], row[2]])

    cur.close()
    con.close()
    
    return all_pods


In [ ]:
qq = selectRows()
print (qq)

In [ ]:
cur = con.cursor()

for eachrow in vv:
    try:
        pod_id = eachrow[0]
        country = eachrow[1]
        feedURL = eachrow[2]
        cur.execute('''UPDATE podcast_master set FeedURL = ? where id = ? AND country = ? ''', (feedURL, pod_id, country))
    except Exception as err:
        pass
con.commit()

cur.close()
con.close()

In [ ]:
def writeEpisodestoDB(descList):
    import sqlite3
    import re
    con = sqlite3.connect('../data/GuidePod.sqlite')

    for eachrow in descList:
        podcastid = eachrow[0]
        podcastdesc = eachrow[1]
        episodes = eachrow[2]
        try:
            cur = con.cursor()
          #  descr = re.sub(r'[^A-Za-z0-9 \.\,\;]+', '',podcastdesc) 

          #  cur.execute('''INSERT INTO podcast_desc values (?,?)''',(podcastid,descr))

            for eachepisode in episodes:
              #  print (descr, podcastid)
                title =eachepisode[0] #re.sub(r'[^A-Za-z0-9 \.\,\;]+', '',eachepisode[0])
                desc =eachepisode[1]  #re.sub(r'[^A-Za-z0-9 \.\,\;]+', '',eachepisode[1])
                dura = eachepisode[2]
                cur.execute('''INSERT INTO episode_detail values (?,?,?,?)''',(podcastid, title, desc, dura ))

        except Exception as err:
            print (err)
            pass
    con.commit()
    con.close()

In [ ]:
writeEpisodestoDB(xx)

In [ ]:
def getEpisode_fromList(listL):
    import xml.etree.ElementTree as ET
    from urllib.request import urlopen
    descList = []  
    for row in listL:
        pod_id = row[0]
        url = row[2]
        print (url)
        try:
            with urlopen(url) as data_file:    
                reviewdata = data_file
                tree = ET.parse(reviewdata)
                root = tree.getroot()
                for c in root.findall('channel'):
                    desc = c.find('description').text
                    episodes = c.findall('item')
                    # <class 'xml.etree.ElementTree.Element'>
                    episodedata = []
                for i in episodes:
                   # type(i) = <class 'xml.etree.ElementTree.Element'>
                   # x = [elem.tag for elem in i.iter() if elem is not i]
                    x1 = i.find('title').text
                    x2 = i.find('description').text
                    x3 = i.find('pubDate').text
                    print (x1,x2,x3)
                    episodedata.append([x1,x2,x3])
            descList.append([pod_id, desc,episodedata])
        except Exception as err:
            print (pod_id, url, err)
            pass
    return descList

In [ ]:
url_list = selectRows()
print (url_list)


In [ ]:
print (type(url_list))

In [ ]:
xx = getEpisode_fromList(url_list)
print (len(xx))
writeEpisodestoDB(xx)

In [ ]:
def getEpisodes(country):
    import xml.etree.ElementTree as ET
    from urllib.request import urlopen
    import sqlite3
    
    con = sqlite3.connect('../data/GuidePod2.sqlite')
    cur = con.cursor()

    sql = '''SELECT  id, FeedURL FROM podcast_master  LIMIT 50 '''
    print (sql)
    rows = cur.execute(sql)
    descList = []    

    for row in rows:
        pod_id = row[0]
        url = row[1]
        print (url)
        try:
            with urlopen(url) as data_file:    
                reviewdata = data_file
                tree = ET.parse(reviewdata)
                root = tree.getroot()
                for c in root.findall('channel'):
                    desc = c.find('description').text
                    episodes = c.findall('item')
                    # <class 'xml.etree.ElementTree.Element'>
                    episodedata = []
                for i in episodes:
                   # type(i) = <class 'xml.etree.ElementTree.Element'>
                   # x = [elem.tag for elem in i.iter() if elem is not i]
                    x1 = i.find('title').text
                    x2 = i.find('description').text
                    x3 = i.find('pubDate').text
                    episodedata.append([x1,x2,x3])
            descList.append([pod_id, desc,episodedata])
        except Exception as err:
            print (pod_id, url, err)

    cur.close()
    con.close()
    return descList


        

In [ ]:
descList = getEpisodes('"au"')
print (len(descList))
writeEpisodestoDB(descList)

In [ ]:
from sqlite3 import dbapi2 as sqlite

def unlock_db(db_filename):
 
    connection = sqlite.connect(db_filename)
    connection.commit()
    connection.close()

unlock_db('../data/GuidePod.sqlite')